In [4]:
setwd("../data/composite_texts/")

In [6]:
####### Read in section definitions from file #######
## These section definitions were generated with the notebook 03_define_sections_R

Q1_sections = read.csv("Q000001_sections.csv", stringsAsFactors = FALSE)
Q39_sections = read.csv("Q000039_sections.csv", stringsAsFactors = FALSE)
Q40_sections = read.csv("Q000040_sections.csv", stringsAsFactors = FALSE)
Q41_sections = read.csv("Q000040_sections.csv", stringsAsFactors = FALSE)
Q42_sections = read.csv("Q000042_sections.csv", stringsAsFactors = FALSE)

In [69]:
# Read in a corpus to analyze

df = read.csv("../pass/ob_lists_wood.csv", stringsAsFactors = FALSE)
head(df)

X,id_line,label,lemma,base,id_text,line,skip,entry
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,P117395,2,0,ŋešed[key]N
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,P117395,3,0,pakud[~tree]N
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,P117395,4,0,raba[clamp]N
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,P117404,2,0,ig[door]N_eren[cedar]N
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,P117404,3,0,ig[door]N_dib[board]N
5,P117404.4,o 3,ig[door]N i[oil]N,{ŋeš}ig i₃,P117404,4,0,ig[door]N_i[oil]N


In [73]:
# for each entry in the corpus, identify which section it belongs to

section_defs = Q39_sections
for (i in 1:nrow(df)) {
    entry = tolower(df$entry[i])
    sections = names(which(sapply(section_defs, function(x) any(x == entry)) == TRUE))
    if(length(sections) ==1) df$section[i] = sections
    df
}
df
# need to cat on if number of matching sections is more than 1


X,id_line,label,lemma,base,id_text,line,skip,entry,section
0,P117395.2,o 1,ŋešed[key]N,{ŋeš}e₃-a,P117395,2,0,ŋešed[key]N,NA
1,P117395.3,o 2,pakud[~tree]N,{ŋeš}pa-kud,P117395,3,0,pakud[~tree]N,NA
2,P117395.4,o 3,raba[clamp]N,{ŋeš}raba,P117395,4,0,raba[clamp]N,NA
3,P117404.2,o 1,ig[door]N eren[cedar]N,{ŋeš}ig {ŋeš}eren,P117404,2,0,ig[door]N_eren[cedar]N,NA
4,P117404.3,o 2,ig[door]N dib[board]N,{ŋeš}ig dib,P117404,3,0,ig[door]N_dib[board]N,door
5,P117404.4,o 3,ig[door]N i[oil]N,{ŋeš}ig i₃,P117404,4,0,ig[door]N_i[oil]N,door
6,P128345.2,o 1,garig[comb]N siki[hair]N,{ŋeš}ga-rig₂ siki,P128345,2,0,garig[comb]N_siki[hair]N,comb
7,P128345.3,o 2,garig[comb]N siki-siki[NA]NA,{ŋeš}ga-rig₂ siki-siki,P128345,3,0,garig[comb]N_siki-siki[NA]NA,NA
8,P128345.4,o 3,garig[comb]N saŋdu[head]N,{ŋeš}ga-rig₂ saŋ-du,P128345,4,0,garig[comb]N_saŋdu[head]N,comb
9,P224980.4,o i 1,gigir[chariot]N,{ŋeš}gigir,P224980,4,0,gigir[chariot]N,chariot


In [77]:
df[2019,]$entry
section_defs[2,]$rudder

[1] "dilim[spoon]N_zigan[rudder]N"

[1] "dilim[spoon]n_zigan[rudder]n"